In [ ]:
from osgeo import gdal, ogr, osr
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pygeoops
import glob
import traceback
from shapely.ops import unary_union
import pandas as pd

# ThreadPoolExecutor：用于I/O密集型任务，使用线程来实现并发。
# ProcessPoolExecutor：用于计算密集型任务，使用进程来实现并发，可以绕过GIL的限制，利用多核处理器。
from concurrent.futures import ThreadPoolExecutor, as_completed,ProcessPoolExecutor

In [ ]:
albers_proj = '+proj=aea +lat_1=18 +lat_2=50 +lat_0=0 +lon_0=105 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'
length = 1263
dir = "E:\\greenland_Campus\\GisDataforChina\\"

# 1.栅格转矢量

## utils

压缩栅格文件

In [ ]:
def compress_raster(input_file, output_file, compress_type='LZW'):
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    driver = gdal.GetDriverByName('GTiff')
    # 设置压缩类型
    options = ['TILED=YES', 'COMPRESS=' + compress_type]
    driver.CreateCopy(output_file, dataset, 0, options, callback=None)
    dataset = None

shp 转成gpkg 

In [ ]:
def ConvertToGeoPackage(shp_file, gpkg_folder_path):
    # 遍历所有Shapefile并转换为GeoPackage

    # 读取Shapefile
    gdf = gpd.read_file(shp_file)
    
    # 构建输出的GeoPackage文件名
    gpkg_file = os.path.join(gpkg_folder_path, os.path.basename(shp_file).replace('.shp', '.gpkg'))
    if os.path.exists(gpkg_file):
        return # 如果GeoPackage文件已经存在，则跳过
    
    # 保存为GeoPackage文件
    gdf.to_file(gpkg_file, driver='GPKG')

简化几何边缘 

In [ ]:
def SimplfyVector(output_vector, tolerance ):
    # 打开刚刚创建的矢量数据源
    vector_ds = ogr.Open(output_vector, update=True)
    if vector_ds is None:
        raise Exception('Unable to open the output vector data source.')

    # 获取图层
    layer = vector_ds.GetLayer()

    # 定义简化容差（以地图单位为单位）

    # 遍历图层中的每个特征并简化几何形状
    for feature in layer:
        if feature.GetField('DN') == 0:
            layer.DeleteFeature(feature.GetFID())
            continue
        geom = feature.GetGeometryRef()
        if(geom==None or not geom.IsValid()):
            layer.DeleteFeature(feature.GetFID())
            continue

        simplified_geom = geom.Simplify(tolerance) # 简化几何形状
        if simplified_geom==None or not simplified_geom.IsValid(): # 如果简化后的几何形状无效，则删除该特征
            layer.DeleteFeature(feature.GetFID()) # 删除无效的要素
        else: # 如果简化后的几何形状有效，则更新该特征
            feature.SetGeometry(simplified_geom)
            layer.SetFeature(feature)

    # 保存并关闭数据源
    vector_ds = None

从栅格变成矢量

In [ ]:
def VectorizeRaster(input_raster, output_vector,tolerance = 1.5e-05):
    ds = gdal.Open(input_raster)
    #lat = ds.GetGeoTransform()[0]

    vector_datasource = ogr.GetDriverByName('ESRI Shapefile').CreateDataSource(output_vector)

    srs = osr.SpatialReference()
    srs.ImportFromWkt(ds.GetProjection())

    layer = vector_datasource.CreateLayer('polygons', srs=srs, geom_type=ogr.wkbPolygon) # 创建一个图层，且使用原图层的投影坐标系
    # 
    layer.CreateField(ogr.FieldDefn('DN', ogr.OFTInteger))
    # 使用 OGR 的 Python 绑定进行矢量化
    gdal.Polygonize(ds.GetRasterBand(1), ds.GetRasterBand(1).GetMaskBand(), layer, 0, callback=None)
    # 清理
    ds = None
    vector_datasource = None
    # 清理后才完成存储
    SimplfyVector(output_vector,tolerance)
    #return lat


## （1）道路

In [ ]:
t = 'road'
output_dir= 'E:\\greenland_Campus\\GisDataforChina\\{}\\'.format(t)

In [ ]:
# 原始版本 
t = 'road'
for i in range(0,1263):
    input_dir = 'H:\\layers\\{}_compressed\\'.format(t)
    output_dir= 'E:\\greenland_Campus\\GisDataforChina\\{}\\'.format(t)

    input_raster = input_dir+'campus_{}.tif'.format(i)
    output_raster= input_dir+'ReClass_{}_{}.tif'.format(t,i)
    output_vector = output_dir+'{}_{}.shp'.format(t,i)
    projection_vector = output_dir+'Proj_{}_{}.shp'.format(t,i) # 投影文件位置

    try:
        # 道路栅格重分类
        ReClassifyRaster(input_raster, output_raster)

        # 道路矢量化为polygon，并简化
        lat=VectorizeRaster(output_raster,output_vector,tolerance = 2e-05)

        # 道路文件投影、删除面积过小的
        zone_number = int((lat+ 180) / 6) + 1 # UTM区带号 = (中央经线 / 6) + 1
        ProjectionVector(output_vector,projection_vector,zone_number= zone_number)

        # 道路中心线
        CenterLine(projection_vector, output_dir+'CenterLine_{}_{}.shp'.format(t,i),0)
    except Exception:
        traceback.print_exc()
        print('error {}'.format(i))
        continue
    if(i%100==0):
        print('finish {}'.format(i))

### 道路：从栅格转矢量

道路栅格重分类（注意压缩）

In [ ]:
def ReClassifyRaster(input_file, output_file):
    # 打开栅格文件
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    band = dataset.GetRasterBand(3)
    raster_array = band.ReadAsArray()

    # 定义重分类规则
    thresholds = {
        0: [0,50],   # 将原始值3重分类为300
        1: [254,255],  # 将原始值1重分类为100
        2: [250,253],  # 将原始值2重分类为200
        3: [170,249],   # 将原始值3重分类为300
        4: [124,169],   # 将原始值3重分类为300
        6: [50,123]   # 将原始值3重分类为300
    }

    # 应用重分类规则
    reclassified_array = np.array(raster_array, copy=True)

    for category, threshold in thresholds.items():
        mask = np.logical_and(raster_array >= threshold[0], raster_array <= threshold[1])
        reclassified_array[mask] = category

    # 创建输出栅格文件
    temp_file = 'temp_reclassified_campus43.tif'.replace('.','_temp.')

    driver = gdal.GetDriverByName('GTiff')
    output_dataset = driver.Create(temp_file, dataset.RasterXSize, dataset.RasterYSize, 1, gdal.GDT_Int16)

    output_dataset.SetGeoTransform(dataset.GetGeoTransform())
    output_dataset.SetProjection(dataset.GetProjection())

    output_band = output_dataset.GetRasterBand(1)
    output_band.WriteArray(reclassified_array)
    output_band.FlushCache()

    # 关闭数据集
    band = None
    dataset = None
    output_band = None
    output_dataset = None

    compress_raster(temp_file, output_file) # 压缩文件大小
    os.remove(temp_file)

In [ ]:
def Raster2Vector(i):
    input_dir = 'H:\\layers\\{}_compressed\\'.format(t)
    input_raster = input_dir+'campus_{}.tif'.format(i)
    output_raster= input_dir+'ReClass_{}_{}.tif'.format(t,i)
    output_vector = output_dir+'{}_{}.shp'.format(t,i)

    try:
        # 道路栅格重分类
        ReClassifyRaster(input_raster, output_raster)

        # 道路矢量化为polygon，并简化，存储为 road_n
        lat=VectorizeRaster(output_raster,output_vector,tolerance = 2e-05)

    except Exception:
        traceback.print_exc()
        print('error {}'.format(i))
        return False
    if(i%100==0):
        print('finish {}'.format(i))

shp转gpkg

In [ ]:
# 获取文件夹中所有的.shp文件
shp_files = glob.glob(os.path.join(output_dir, '*.shp'))

with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(ConvertToGeoPackage, shp_file, output_dir) for shp_file in shp_files]

    # 可选：等待所有任务完成
    for future in futures:
        future.result()  # 这将引发任何在任务中发生的异常

###  提取道路中心线（先投影，增加长度字段）

In [ ]:
def CenterLine(input_shp, output_shp,min_length=20,buffer=4):
    gdf = gpd.read_file(input_shp) 
    # 缓冲区合并
    gdf['geometry'] = gdf['geometry'].buffer(buffer) # 原始

    # 合并图形
    merged_gdf = gdf.dissolve(aggfunc='first') 
    # 生成中心线
    merged_gdf.geometry = pygeoops.centerline(merged_gdf.geometry, simplifytolerance=-0.4)
    merged_gdf=merged_gdf.explode()

    merged_gdf.reset_index(inplace=True,drop=True)
    merged_gdf.drop(columns=['DN'], inplace=True)
    merged_gdf['index'] = merged_gdf.index

    merged_gdf['length'] = merged_gdf['geometry'].length

    filtered_gdf = merged_gdf[merged_gdf['length'] > min_length]

    gdf.to_file(input_shp)
    filtered_gdf.to_file(output_shp)

运行

In [ ]:
def Polygon2Line(i):
    output_vector = output_dir+'{}_{}.gpkg'.format(t,i)
    projection_vector = output_dir+'Proj_{}_{}.gpkg'.format(t,i) # 投影文件位置

    polygon_gdf = gpd.read_file(output_vector)

    try:
        # 道路文件投影
        zone_number = int((lat+ 180) / 6) + 1 # UTM区带号 = (中央经线 / 6) + 1
        proj_gdf = polygon_gdf.to_crs("EPSG:{}".format(32600+zone_number))
        # 删除面积过小的
        proj_gdf['area'] = proj_gdf.area
        proj_gdf.filter(proj_gdf['DN']!=0 & proj_gdf['area']>5,inplace=True)
        proj_gdf.to_file(projection_vector)

        # 道路中心线
        CenterLine(proj_gdf, output_dir+'CenterLine_{}_{}.gpkg'.format(t,i),0)

    except Exception:
        traceback.print_exc()
        print('error {}'.format(i))
        return False
    if(i%100==0):
        print('finish {}'.format(i))
        
# 批量运行      
with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(Polygon2Line, i) for i in range(length)]

    # 可选：等待所有任务完成
    for future in futures:
        future.result()  # 这将引发任何在任务中发生的异常

#### 处理提取中心线时的异常道路

In [ ]:
i=60  # 60 117，407，595
t = 'road'
output_dir= 'E:\\greenland_Campus\\GisDataforChina\\{}_gpkg\\'.format(t)
input_shp = output_dir+'Proj_{}_{}.gpkg'.format(t,i) # 投影文件位置
output_shp = output_dir+'CenterLine_{}_{}.gpkg'.format(t,i)

buffer=0
gdf = gpd.read_file(input_shp) 

In [ ]:
gdf.loc[gdf['DN'] == 3, 'DN'] = 4
gdf.loc[gdf['DN'] == 6, 'DN'] = 4
# gdf.loc[gdf['DN'] == 2, 'DN'] = 4
gdf.loc[gdf['DN'] == 2, 'DN'] = 1

In [ ]:
gdf

In [ ]:
# 合并图形
merged_gdf = gdf.dissolve(by='DN',aggfunc='first') 
merged_gdf

In [ ]:
# 生成中心线
merged_gdf.geometry = pygeoops.centerline(merged_gdf.geometry, simplifytolerance=-0.4)
merged_gdf=merged_gdf.explode()

merged_gdf.reset_index(inplace=True,drop=True) 
# merged_gdf.drop(columns=['DN'], inplace=True)
merged_gdf['index'] = merged_gdf.index

merged_gdf['length'] = merged_gdf['geometry'].length

#filtered_gdf = merged_gdf[merged_gdf['length'] > min_length]

#gdf.to_file(input_shp)
merged_gdf.to_file(output_shp)

In [ ]:
list = [60,117,407,595]
for i in list:
    df = gpd.read_file(dir_road+'CenterLine_{}_{}.gpkg'.format(t,i))
    df = df.explode()
    df.to_file(dir_road+'CenterLine_{}_{}.gpkg'.format(t,i))

## （2）多边形

In [ ]:
# 处理多边形并批量运行
def ProcessPolygon(types,i):
    for t in types:
        output_dir= 'E:\\greenland_Campus\\GisDataforChina\\{}\\'.format(t)
        input_dir = 'H:\\layers\\{}_compressed\\'.format(t)

        input_raster = input_dir+'campus_{}.tif'.format(i)
        output_vector = output_dir+'{}_{}.shp'.format(t,i)
        
        try:
            # 矢量化
            # VectorizeRaster(input_raster, output_vector) # 矢量化为polygon
            lat = VectorizeRaster(input_raster, output_vector) # 矢量化为polygon

            # 投影、删除无用的要素
            zone_number = int((lat+ 180) / 6) + 1 # UTM区带号 = (中央经线 / 6) + 1
            gdf = gpd.read_file(output_vector)
            gdf = gdf.to_crs("EPSG:{}".format(32600+zone_number))
            projection_vector = output_dir+'Proj_{}_{}.shp'.format(t,i) # 投影文件位置

            proj_gdf = gpd.read_file(output_vector)
            proj_gdf['area'] = proj_gdf.area
            proj_gdf.filter(proj_gdf['DN']!=0 & proj_gdf['area']>5,inplace=True)
            proj_gdf.to_file(projection_vector)

        except Exception:
                traceback.print_exc()
                print('error {}_{}'.format(t,i))
                continue
    if(i%100==0):
        print('finish {}'.format(i))


types=['buildings','green','river']

# 批量处理
with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(types, i) for i in range(length)]

    # 可选：等待所有任务完成
    for future in futures:
        future.result()  # 这将引发任何在任务中发生的异常

绿地文件合并green和river

In [ ]:
shp_folder_path = "E:\\greenland_Campus\\GisDataforChina\\green"

# 设置输出GeoPackage文件夹路径
gpkg_folder_path ="E:\\greenland_Campus\\GisDataforChina\\green_gpkg"

# 获取文件夹中所有的.shp文件
shp_files = glob.glob(os.path.join(shp_folder_path, '*.shp'))

# 定义函数将Shapefile转换为GeoPackage,并合并green和river
def ConvertToGPKGforGreen(shp_file):
    # 读取Shapefile
    gdf = gpd.read_file(shp_file)
    
    gdf1 = gpd.read_file(shp_file.replace('green', 'river').replace('riverland', 'greenland') )# 读取river
    gdf1['type'] = 'river'
    gdf['type'] = 'green'
    gdf_out = pd.concat([gdf, gdf1],ignore_index=True)
    
    # 构建输出的GeoPackage文件名
    gpkg_file = os.path.join(gpkg_folder_path, os.path.basename(shp_file).replace('.shp', '.gpkg'))
    if os.path.exists(gpkg_file):
        return
    
    # 保存为GeoPackage文件
    gdf_out.to_file(gpkg_file, driver='GPKG')

# 批量处理
with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(ConvertToGPKGforGreen, shp_file) for shp_file in shp_files]

    # 可选：等待所有任务完成
    for future in futures:
        future.result()  # 这将引发任何在任务中发生的异常